# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [35]:
# import libraries
import numpy as np
import pandas as pd
from sqlalchemy import create_engine

In [36]:
# load data from database
engine = create_engine('sqlite:///MessagesClean.db')
df = pd.read_sql("SELECT * FROM messages_clean", engine)
X = df['message']
Y = df.drop(['id', 'message', 'original', 'genre'], axis = 1)

In [37]:
df

,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,aid_centers,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,7,Is the Hurricane over or is it not over,Cyclone nan fini osinon li pa fini,direct,1,0,0,1,0,0,...,0,0,1,0,1,0,0,0,0,0
2,8,Looking for someone but no name,"Patnm, di Maryani relem pou li banm nouvel li ...",direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,9,UN reports Leogane 80-90 destroyed. Only Hospi...,UN reports Leogane 80-90 destroyed. Only Hospi...,direct,1,1,0,1,0,1,...,0,0,0,0,0,0,0,0,0,0
4,12,"says: west side of Haiti, rest of the country ...",facade ouest d Haiti et le reste du pays aujou...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26108,30261,The training demonstrated how to enhance micro...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26109,30262,A suitable candidate has been selected and OCH...,None,news,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26110,30263,"Proshika, operating in Cox's Bazar municipalit...",None,news,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
26111,30264,"Some 2,000 women protesting against the conduc...",None,news,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [38]:
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
nltk.download(['punkt', 'stopwords', 'wordnet'])

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ROLD003\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ROLD003\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ROLD003\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [39]:
def tokenize(text):
    text = re.sub(r'[^a-zA-Z0-9}]', ' ', text.lower())
    tokens = word_tokenize(text)
    stop_words = stopwords.words('english')
    lemmatizer =  WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return tokens

In [40]:
tokenize(df['message'][0])

['weather', 'update', 'cold', 'front', 'cuba', 'could', 'pas', 'haiti']

In [41]:
df['message'][0]

'Weather update - a cold front from Cuba that could pass over Haiti'

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [42]:
from sklearn.datasets import make_multilabel_classification
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline

In [43]:
pipeline = Pipeline([('vetc', CountVectorizer(tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [44]:
from sklearn.model_selection import train_test_split

In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, random_state=43)

np.random.seed(20)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vetc',
                 CountVectorizer(tokenizer=<function tokenize at 0x0000018C19794438>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [52]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, make_scorer

In [47]:
y_pred = pipeline.predict(X_train)
y_true = np.array(y_train)
target_names = list(Y.columns.values)

In [48]:
y_pred

array([[1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [49]:
y_true

array([[1, 0, 0, ..., 1, 0, 0],
       [1, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [1, 1, 0, ..., 0, 0, 1],
       [1, 1, 0, ..., 0, 0, 0]], dtype=int64)

In [50]:
y_pred.size, y_true.size

(705024, 705024)

In [51]:
target_names

['related',
 'request',
 'offer',
 'aid_related',
 'medical_help',
 'medical_products',
 'search_and_rescue',
 'security',
 'military',
 'child_alone',
 'water',
 'food',
 'shelter',
 'clothing',
 'money',
 'missing_people',
 'refugees',
 'death',
 'other_aid',
 'infrastructure_related',
 'transport',
 'buildings',
 'electricity',
 'tools',
 'hospitals',
 'shops',
 'aid_centers',
 'other_infrastructure',
 'weather_related',
 'floods',
 'storm',
 'fire',
 'earthquake',
 'cold',
 'other_weather',
 'direct_report']

Considering it is a multiclass-multioutput, I have to create a for loop to display the results in a DataFrame.

In [67]:
def display_results(y_test, y_pred, col_names):
    results = []
    for i in range(len(col_names)):
        accuracy = accuracy_score(y_test[:, i], y_pred[:, i])
        precision = precision_score(y_test[:, i], y_pred[:, i], average='weighted')
        recall = recall_score(y_test[:, i], y_pred[:, i], average='weighted')
        f1 = f1_score(y_test[:, i], y_pred[:, i], average='weighted')
        results.append([accuracy, precision, recall, f1])
    
    results = np.array(results)
    results_df = pd.DataFrame(data=results, index=col_names, columns=['% Accuracy', '% Precision', '% Recall', '% F1'])  
    return results_df

Below, I will evaluate the results of the train set:

In [68]:
y_pred = pipeline.predict(X_train)
col_names = list(Y.columns.values)
results_train = pd.DataFrame(display_results(np.array(y_train), y_pred, col_names))
results_train

,% Accuracy,% Precision,% Recall,% F1
related,0.998417,0.998418,0.998417,0.998416
request,0.999234,0.999234,0.999234,0.999234
offer,1.000000,1.000000,1.000000,1.000000
aid_related,0.999183,0.999183,0.999183,0.999183
medical_help,0.999745,0.999745,0.999745,0.999745
medical_products,0.999847,0.999847,0.999847,0.999847
search_and_rescue,0.999847,0.999847,0.999847,0.999847
security,0.999847,0.999847,0.999847,0.999847
military,0.999949,0.999949,0.999949,0.999949
child_alone,1.000000,1.000000,1.000000,1.000000


In [69]:
results_train.describe()

,% Accuracy,% Precision,% Recall,% F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.999769,0.999769,0.999769,0.999769
std,0.000322,0.000322,0.000322,0.000322
min,0.998417,0.998418,0.998417,0.998416
25%,0.999745,0.999745,0.999745,0.999745
50%,0.999898,0.999898,0.999898,0.999898
75%,0.999949,0.999949,0.999949,0.999949
max,1.000000,1.000000,1.000000,1.000000


And the results for the test set:

In [70]:
y_pred2 = pipeline.predict(X_test)
col_names = list(Y.columns.values)
results_test = pd.DataFrame(display_results(np.array(y_test), y_pred2, col_names))
results_test

C:\Users\ROLD003\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1221: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


,% Accuracy,% Precision,% Recall,% F1
related,0.814214,0.801856,0.814214,0.795510
request,0.901363,0.895849,0.901363,0.891726
offer,0.995558,0.991136,0.995558,0.993342
aid_related,0.780824,0.779288,0.780824,0.779321
medical_help,0.923725,0.900738,0.923725,0.894884
medical_products,0.953745,0.942880,0.953745,0.934445
search_and_rescue,0.973197,0.971254,0.973197,0.961364
security,0.983612,0.968090,0.983612,0.975789
military,0.969521,0.966442,0.969521,0.956544
child_alone,1.000000,1.000000,1.000000,1.000000


In [71]:
results_test.describe()

,% Accuracy,% Precision,% Recall,% F1
count,36.000000,36.000000,36.000000,36.000000
mean,0.948861,0.938138,0.948861,0.936996
std,0.050854,0.054553,0.050854,0.055490
min,0.780824,0.779288,0.780824,0.779321
25%,0.938620,0.925271,0.938620,0.923413
50%,0.961097,0.951606,0.961097,0.946948
75%,0.984339,0.977821,0.984339,0.977166
max,1.000000,1.000000,1.000000,1.000000


### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
parameters = 

cv = 

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.